In [3]:
import requests
from master import headers
import json
import pandas as pd

url = "https://pcmap-api.place.naver.com/graphql"

# headers = {'User-Agent' : '개인 user-agent', 'Content-Type': 'application/json', 'accept': '*/*', 'accept-encoding': 'gzip, deflate, br', 'accept-language': 'ko'}
headers['Content-Type'] = "application/json"
headers['accept'] = '*/*'
headers['accept-encoding'] = 'gzip, deflate, br'
headers['accept-language'] = 'ko'
    

In [4]:
r.json().keys()

NameError: name 'r' is not defined

In [5]:
r.json().values()

for k,v in r.json().items():
    print(f"key: {k}, value: {v} \n")

NameError: name 'r' is not defined

In [6]:
import pyproj
epsg3857 = pyproj.Proj(init='epsg:3857')
wgs84 = pyproj.Proj(init='epsg:4326')
pyproj.transform(epsg3857,wgs84,14127898.0825082,4515937.3711204)

c:\Anaconda\envs\study\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Anaconda\envs\study\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\강건\AppData\Local\Temp/ipykernel_20092/943088905.py:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  pyproj.transform

(126.91306779999996, 37.54920139999987)

In [7]:
import time
places = {}
for j in range(1,30):
    r = requests.get(url=f'https://map.naver.com/v5/api/search?caller=pcweb&query=%EC%9D%8C%EC%8B%9D%EC%A0%90&type=all&searchCoord=126.91306779999996;37.54920139999987&page={j}&displayCount=100&isPlaceRecommendationReplace=true&lang=ko', headers=headers)
    if 'error' in r.json().keys():
        print(r.json()['error']['displayMsg'])
        break
    places[j] = r.json()['result']['place']['list']
    time.sleep(0.5)
places

잘못된 요청입니다.


{1: [{'index': '0',
   'rank': '1',
   'id': '1266638776',
   'name': '합정공감',
   'tel': '02-333-4733',
   'isCallLink': False,
   'virtualTel': '0507-1423-4733',
   'ppc': '1',
   'category': ['술집', '이자카야'],
   'categoryPath': [['220036', '220051', '220556', '1003251'],
    ['1004760', '1004380', '1002043']],
   'address': '서울특별시 마포구 서교동 396-33 2F',
   'roadAddress': '서울특별시 마포구 양화로6길 49 2F',
   'abbrAddress': '서교동 396-33 2F',
   'shortAddress': ['서울 마포구', '양화로6길 49', '2F'],
   'display': '합정공감',
   'telDisplay': '02-333-4733',
   'context': [],
   'reviewCount': 314,
   'placeReviewCount': 575,
   'ktCallMd': '0c0abb08f58ebb0f3890af8c31558ff5',
   'coupon': '0',
   'thumUrl': 'https://ldb-phinf.pstatic.net/20220415_206/1649997530821L2Hsh_JPEG/11.jpg',
   'thumUrls': ['https://ldb-phinf.pstatic.net/20220415_206/1649997530821L2Hsh_JPEG/11.jpg',
    'https://ldb-phinf.pstatic.net/20220114_205/1642142763654D3liI_JPEG/IM_00092_B.jpg',
    'https://ldb-phinf.pstatic.net/20220114_255/16421428

In [6]:
len(places)

3

In [8]:
# for k,v in places[1][50].items():
#     print(f"key: {k}, value: {v}")

# bus_id = [{k:v} for k,v in zip(places[1][0]['name'],places[1][0]['id'])]
# bus_id

place_id_dict = {}
for i in places.values():
    for j in i:
        place_id_dict[j['name']] = j['id']

place_id_dict

{'합정공감': '1266638776',
 '우동 카덴': '34393996',
 '오레노라멘 본점': '814481697',
 '스케줄합정': '1836765337',
 '멕시코식당': '1700484018',
 '모센즈스위트': '1021945903',
 '교다이야': '36864685',
 '도식화': '1578071453',
 '합정고깃집': '38276889',
 '동두천솥뚜껑삼겹살 합정점': '1353043108',
 '테이스티버거': '1935683021',
 '피오니': '19866326',
 '우와': '56228015',
 '최강금돈까스': '1508260607',
 '어반플랜트 합정': '1593016664',
 '야키토리 나루토': '1294694156',
 '카와카츠 합정점': '1152661518',
 '미도인 홍대': '1436718221',
 '아우룸': '1476397400',
 '포비 베이직': '713289996',
 '함반': '1431631412',
 '미나리': '1661787502',
 '시오': '1605124064',
 '썬키친': '1844126724',
 '히츠지야': '181824763',
 '육지': '1287775548',
 '피자네버슬립스': '1554706925',
 '저스티나': '1580543297',
 '텍사스바베큐레스토랑': '1831597455',
 '금별맥주 홍대직영점': '1456505357',
 '오디티': '1922515332',
 '합정티라미수': '1132863024',
 '우마담': '1658726759',
 '나성타코': '1185189754',
 '스테이크 슈퍼': '1481535598',
 '감성타코 합정점': '625311806',
 '신미경홍대닭갈비': '32572240',
 '또보겠지떡볶이집 호호시스터점': '37663561',
 '빠넬로': '33707338',
 '동무밥상': '37387671',
 '당고집': '13122422',
 '츠키젠': '1172910030'

In [9]:
place_info = []
for idx,place in enumerate(place_id_dict.values()):
    place_info.append(requests.get(url=f'https://map.naver.com/v5/api/sites/summary/{place}?lang=ko',headers=headers).json())
    if idx == 20:
        break
    time.sleep(0.5)

In [19]:
place_info[0].keys()

dict_keys(['id', 'isSite', 'name', 'type', 'rCode', 'x', 'y', 'posExact', 'address', 'fullAddress', 'addressAbbr', 'roadAddr', 'fullRoadAddress', 'phone', 'isCallLink', 'description', 'way', 'keywords', 'hasNaverTalktalkUrl', 'naverTalktalkUrl', 'adult', 'endPageUrl', 'mobileEndPageUrl', 'urlList', 'ktCallMd', 'ppc', 'images', 'imageURL', 'imageModDate', 'displayCategory', 'category', 'categories', 'categoryPaths', 'bizHour', 'bizhourInfo', 'options', 'menus', 'menuImages', 'previewImages', 'streetPanorama', 'skyPanorama', 'insidePanorama', 'interiorPanorama', 'indoorPanorama', 'entranceCoords', 'theme', 'hasNaverBooking', 'naverBookingUrl', 'naverbookingId', 'petrolInfo', 'michelinGuide', 'broadcastInfo', 'marker', 'markerSelected', 'datalab', 'reviewCount', 'dynamicData', 'markerLabel', 'isParkingSupported'])

In [23]:
options = pd.DataFrame(place_info[0]['options'])
options['bus_id'] = place_info[0]['id']

for idx in range(1,len(place_info)):
    df = pd.DataFrame(place_info[idx]['options'])
    df['bus_id'] = place_info[idx]['id']
    options = pd.concat([options,df])

options.reset_index(drop=True,inplace=True)

options



,id,name,isCheck,order,iconURL,desc,bus_id
0,13.0,단체석,1,0.0,http://static.naver.net/maps2/ic_endinfo13.png,,1266638776
1,4.0,포장,1,3.0,http://static.naver.net/maps2/ic_endinfo4.png,,1266638776
2,1.0,예약,1,6.0,http://static.naver.net/maps2/ic_endinfo1.png,,1266638776
3,7.0,무선 인터넷,1,7.0,http://static.naver.net/maps2/ic_endinfo7.png,,1266638776
4,8.0,남/녀 화장실 구분,1,10.0,http://static.naver.net/maps2/ic_endinfo8.png,,1266638776
5,39.0,제로페이,1,15.0,http://static.naver.net/maps2/ic_endinfo39.png,,1266638776
6,13.0,단체석,1,0.0,http://static.naver.net/maps2/ic_endinfo13.png,,1836765337
7,4.0,포장,1,3.0,http://static.naver.net/maps2/ic_endinfo4.png,,1836765337
8,1.0,예약,1,6.0,http://static.naver.net/maps2/ic_endinfo1.png,,1836765337
9,7.0,무선 인터넷,1,7.0,http://static.naver.net/maps2/ic_endinfo7.png,,1836765337


In [11]:
place_profile = pd.DataFrame(place_info)[['id','name','x','y','fullAddress','phone','description','images','imageURL','categories','bizhourInfo','menus','reviewCount']]
place_profile

,id,name,x,y,fullAddress,phone,description,images,imageURL,categories,bizhourInfo,menus,reviewCount
0,34393996,우동 카덴,126.915012,37.551708,서울특별시 마포구 서교동 391-5,02-6463-6362,월~금 오전 : 11:30 ~ 14:40 \n 오후 : 17:30 ~...,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20150901_299/144...,"[일식, 우동,소바]",평일 11:30~14:30 점심 마지막주문 14:40 | 평일 17:30~21:30...,"[{'name': '가케우동', 'price': '8,000원', 'isRecomm...",2152
1,814481697,오레노라멘 본점,126.917315,37.547478,서울특별시 마포구 합정동 361-1,02-322-3539,영업시간 11:00 ~ 21:00 (L.o 20:30)\n\n '연중무휴' 휴무시 ...,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20181030_171/154...,"[일식, 일본식라면]",매일 11:30~21:00 재료준비시간 14:40~17:00,"[{'name': '토리빠이탄 라멘', 'price': '10,000원', 'isR...",1773
2,1836765337,스케줄합정,126.916055,37.549073,서울특별시 마포구 서교동 394-17 1층,02-322-2622,<스케줄 합정점은 상단의 링크를 통해 캐치테이블 예약이 가능합니다.>\n\n<스케줄...,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20220415_243/164...,"[음식점, 양식]",평일 11:00~01:00 No Breaktime ㅣ정부 방침에 따른 영업시간 조정,"[{'name': '스케줄 김치 볶음밥', 'price': '17,000원', 'i...",1175
3,1578071453,도식화,126.921792,37.549785,서울특별시 마포구 서교동 409-19 1층,010-6353-7937,우리는 그림을 그립니다\n\n우리가 그리는 그림은\n종이와 물감이 필요하지 않습니다...,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20220704_12/1656...,"[음식점, 카페,디저트]",None,"[{'name': '간장들깨 마들렌', 'price': '4,300원', 'isRe...",360
4,1266638776,합정공감,126.917017,37.548927,서울특별시 마포구 서교동 396-33 2F,02-333-4733,프리미엄 이자카야 공감 합정점은 15년경력의 일식쉐프가 만드는 수제요리 전문점입니다...,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20220415_206/164...,"[술집, 이자카야]",매일 17:00~02:00 코로나19 정부 방침에 따른 영업시간 조정 | 공휴일 1...,"[{'name': '공감 제철 특 사시미', 'price': '62,000원', '...",268
5,1021945903,모센즈스위트,126.920848,37.549835,서울특별시 마포구 서교동 403-3 1층 102호 모센즈스위트,070-7543-7354,"안녕하세요, 아랍 디저트 카페 '모센즈스위트'입니다.\n\n중동 지역의 요르단에는 ...","[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20220202_166/164...,"[음식점, 카페,디저트]",매일 12:00~22:00,"[{'name': '쿠나파', 'price': '6,000원', 'isRecomme...",577
6,1700484018,멕시코식당,126.914578,37.547504,서울특별시 마포구 합정동 366-7,02-336-3666,신선한 재료로 정직하게 만든 멕시코음식을 빠르게 제공합니다,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20210319_206/161...,"[음식점, 멕시코,남미음식]",매일 11:30~22:00 오후3시~4시 재료준비시간,"[{'name': '치미창가', 'price': '15,500원', 'isRecom...",702
7,36864685,교다이야,126.913200,37.546920,서울특별시 마포구 합정동 370-8 합정빌딩 1층,02-2654-2645,,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20190516_90/1557...,"[일식, 우동,소바]",매일 11:00~20:30 Break time 15:00~17:00 | 월요일 휴무,"[{'name': '가케우동', 'price': '8,000원', 'isRecomm...",1333
8,19866326,피오니,126.919829,37.550071,서울특별시 마포구 서교동 395-78,02-333-5325,딸기케이크 및 디저트와 음료를 판매합니다.,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20150831_255/144...,"[카페,디저트, 케이크전문]","매일 12:00~22:00 마지막 오더 21:00 (추석당일,전일/ 설당일,전일 휴무)","[{'name': '딸기빙수', 'price': '11,000원', 'isRecom...",2496
9,56228015,우와,126.922223,37.551714,서울특별시 마포구 서교동 364-10,070-7778-9500,안녕하세요 .\n\n오코노미야끼 와 야끼소바 각종 테판야끼를 전문으로 하고 있는 雨...,"[{'type': 'DRT', 'groupName': '기본 정보', 'number...",https://ldb-phinf.pstatic.net/20180124_31/1516...,"[일식, 일식당]",월요일 12:00~22:00 라스트오더 21:00 | 화요일 12:00~22:00 ...,"[{'name': '새우토마토오코노미야끼', 'price': '17,000원', '...",1820


In [15]:
# images 테이블
images_list = []
for i in range(len(place_info)):
    for j in range(len(place_info[i]['images'])):
        place_info[i]['images'][j]['bus_id'] = place_info[i]['id']
    images_list.append(place_info[i]['images'])

df = pd.DataFrame(images_list[0])

for i in range(1,len(images_list)):
    df = pd.concat([df,pd.DataFrame(images_list[i])])

df = df.reset_index(drop=True)[['bus_id','number','url','modDate']]
df

,bus_id,number,url,modDate
0,34393996,1.0,https://ldb-phinf.pstatic.net/20150901_299/144...,2020-09-29 05:04:38
1,34393996,2.0,https://ldb-phinf.pstatic.net/20150901_203/144...,2020-09-29 05:04:38
2,34393996,3.0,https://ldb-phinf.pstatic.net/20150901_96/1441...,2020-09-29 05:04:38
3,34393996,4.0,https://ldb-phinf.pstatic.net/20150901_186/144...,2020-09-29 05:04:38
4,34393996,5.0,https://ldb-phinf.pstatic.net/20150901_144/144...,2020-09-29 05:04:38
...,...,...,...,...
454,38276889,28.0,https://ldb-phinf.pstatic.net/20201217_271/160...,2020-12-17 19:44:45
455,38276889,29.0,https://ldb-phinf.pstatic.net/20201217_262/160...,2020-12-17 19:44:45
456,38276889,30.0,https://ldb-phinf.pstatic.net/20201217_143/160...,2020-12-17 19:44:45
457,38276889,31.0,https://ldb-phinf.pstatic.net/20201217_72/1608...,2020-12-17 19:44:45


In [16]:
# menus
menus = pd.DataFrame(place_profile['menus'][0])
menus['bus_id'] = place_profile['id'][0]

for i in range(1,len(place_profile)):
    df = pd.DataFrame(place_profile['menus'][i])
    df['bus_id'] = place_profile['id'][i]
    menus = pd.concat([menus,df])

menus = menus[['bus_id','name','price']].reset_index(drop=True)
menus

,bus_id,name,price
0,34393996,가케우동,"8,000원"
1,34393996,덴뿌라우동,"11,000원"
2,34393996,붓가케우동,"8,500원"
3,34393996,유부우동,"10,000원"
4,34393996,모시조개 우동,"11,000원"
...,...,...,...
294,38276889,토시살,"19,000원"
295,38276889,모듬A,"40,000원"
296,38276889,모듬B,"45,000원"
297,38276889,모듬C,"52,000원"


In [14]:
# 식당 기본 상세정보
place_default_profile = place_profile[['id','name','x','y','fullAddress','phone','description','imageURL','bizhourInfo','reviewCount']]
place_default_profile

,id,name,x,y,fullAddress,phone,description,imageURL,bizhourInfo,reviewCount
0,34393996,우동 카덴,126.915012,37.551708,서울특별시 마포구 서교동 391-5,02-6463-6362,월~금 오전 : 11:30 ~ 14:40 \n 오후 : 17:30 ~...,https://ldb-phinf.pstatic.net/20150901_299/144...,평일 11:30~14:30 점심 마지막주문 14:40 | 평일 17:30~21:30...,2152
1,814481697,오레노라멘 본점,126.917315,37.547478,서울특별시 마포구 합정동 361-1,02-322-3539,영업시간 11:00 ~ 21:00 (L.o 20:30)\n\n '연중무휴' 휴무시 ...,https://ldb-phinf.pstatic.net/20181030_171/154...,매일 11:30~21:00 재료준비시간 14:40~17:00,1773
2,1836765337,스케줄합정,126.916055,37.549073,서울특별시 마포구 서교동 394-17 1층,02-322-2622,<스케줄 합정점은 상단의 링크를 통해 캐치테이블 예약이 가능합니다.>\n\n<스케줄...,https://ldb-phinf.pstatic.net/20220415_243/164...,평일 11:00~01:00 No Breaktime ㅣ정부 방침에 따른 영업시간 조정,1175
3,1578071453,도식화,126.921792,37.549785,서울특별시 마포구 서교동 409-19 1층,010-6353-7937,우리는 그림을 그립니다\n\n우리가 그리는 그림은\n종이와 물감이 필요하지 않습니다...,https://ldb-phinf.pstatic.net/20220704_12/1656...,None,360
4,1266638776,합정공감,126.917017,37.548927,서울특별시 마포구 서교동 396-33 2F,02-333-4733,프리미엄 이자카야 공감 합정점은 15년경력의 일식쉐프가 만드는 수제요리 전문점입니다...,https://ldb-phinf.pstatic.net/20220415_206/164...,매일 17:00~02:00 코로나19 정부 방침에 따른 영업시간 조정 | 공휴일 1...,268
5,1021945903,모센즈스위트,126.920848,37.549835,서울특별시 마포구 서교동 403-3 1층 102호 모센즈스위트,070-7543-7354,"안녕하세요, 아랍 디저트 카페 '모센즈스위트'입니다.\n\n중동 지역의 요르단에는 ...",https://ldb-phinf.pstatic.net/20220202_166/164...,매일 12:00~22:00,577
6,1700484018,멕시코식당,126.914578,37.547504,서울특별시 마포구 합정동 366-7,02-336-3666,신선한 재료로 정직하게 만든 멕시코음식을 빠르게 제공합니다,https://ldb-phinf.pstatic.net/20210319_206/161...,매일 11:30~22:00 오후3시~4시 재료준비시간,702
7,36864685,교다이야,126.913200,37.546920,서울특별시 마포구 합정동 370-8 합정빌딩 1층,02-2654-2645,,https://ldb-phinf.pstatic.net/20190516_90/1557...,매일 11:00~20:30 Break time 15:00~17:00 | 월요일 휴무,1333
8,19866326,피오니,126.919829,37.550071,서울특별시 마포구 서교동 395-78,02-333-5325,딸기케이크 및 디저트와 음료를 판매합니다.,https://ldb-phinf.pstatic.net/20150831_255/144...,"매일 12:00~22:00 마지막 오더 21:00 (추석당일,전일/ 설당일,전일 휴무)",2496
9,56228015,우와,126.922223,37.551714,서울특별시 마포구 서교동 364-10,070-7778-9500,안녕하세요 .\n\n오코노미야끼 와 야끼소바 각종 테판야끼를 전문으로 하고 있는 雨...,https://ldb-phinf.pstatic.net/20180124_31/1516...,월요일 12:00~22:00 라스트오더 21:00 | 화요일 12:00~22:00 ...,1820


In [98]:
import pymysql

place_test = pd.DataFrame(place_info)[['id','name','x','y','fullAddress']]

db = pymysql.connect(host='127.0.0.1',port=3306,user='root',passwd='1234',db='testdb',charset='utf8')


In [303]:
cursor = db.cursor()

sql = """
    CREATE TABLE IF NOT EXISTS place_info (
        id INT NOT NULL AUTO_INCREMENT,
        name VARCHAR(40) NOT NULL,
        x_position FLOAT,
        y_position FLOAT,
        PRIMARY KEY(id)
    )
"""
cursor.execute(sql)

cursor.execute("SHOW TABLES")

# cursor.close()

4

In [306]:
for _, row in place_profile.iterrows():
    # print(row)
    sql  = f"""
        INSERT INTO place_info (name,x_position,y_position) VALUES ({row['id']},{row['x']},{row['y']})
        """
    print(sql)
    cursor.execute(sql)

db.commit()
    


        INSERT INTO place_info (name,x_position,y_position) VALUES (1633977299,126.3332056,33.4105799)
        

        INSERT INTO place_info (name,x_position,y_position) VALUES (1439356450,126.3392341,33.4019981)
        

        INSERT INTO place_info (name,x_position,y_position) VALUES (1449299326,126.33846166500638,33.40327431533632)
        

        INSERT INTO place_info (name,x_position,y_position) VALUES (1051333889,126.31842982221268,33.364894947887976)
        

        INSERT INTO place_info (name,x_position,y_position) VALUES (933405444,126.3143903,33.415685)
        

        INSERT INTO place_info (name,x_position,y_position) VALUES (19873713,126.28425203203561,33.383904880598955)
        
